<a href="https://colab.research.google.com/github/WesleyAldridge/HW2_MachineLearning/blob/master/HW2_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW2 - Convolutional Neural Network For CIFAR10 Data Set

####Instructions from professor:

"The goal of this homework is to create a convolutional neural network for the CIFAR10 data set.

Make sure that the data is divided into:

- training set,
- validation set
- test set.

Use the training set to train your neural networks. Evaluate their performance on the validation data set.

After trying several different architectures, choose the one that performs best of the validation set.

Train this final architecture on the data from the training set and validation set and evaluate its performance on the test set."

## Loading the CIFAR10 data set

**train_images.shape = (50000, 32, 32, 3)**

**test_images.shape  = (10000, 32, 32, 3) **

50,000 train_images of shape 32x32 pixels, made up of 3 channels (R,G,B)

10,000 test_images of shape 32x32 pixels, made up of 3 channels (R,G,B)

In [103]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.datasets import cifar10
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

training_set = train_images
test_set = test_images

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

training_set.shape, test_set.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

## Making CNN

Example CNN from class notes:

Input Images > Conv2d (ReLU) > MaxPool > Conv2d (ReLU) > MaxPool > Fully Connected > Fully Connected (output) 

This is two convolution modules (convolution(ReLU) + pooling) for feature extraction, and two fully connected layers for classification.

In [0]:
CNN = models.Sequential()

# "To start, the CNN receives an input feature map: a 3-dimensional matrix, where
# the size of the first two dimensions corresponds to the length and width of the
# images in pixels, and the size of the third dimension is 3 (corresponding to the
# 3 channels of a color image: red, green, and blue)." - class notes

# "A convolution extracts tiles of the input feature map, and applies filters to them
# to compute new features, producing an output feature map, or convolved feature
# (which may have a different size and depth than the input feature map)
# Convolutions are defined by two parameters:
# Size of the tiles that are extracted (typically 3x3 or 5x5 pixels).
# The depth of the output feature map, which corresponds to the number of filters
# that are applied." - class notes

# "After each convolution operation, the CNN applies a Rectified Linear Unit (ReLU)
# transformation to the convolved feature, in order to introduce nonlinearity into
# the model" - class notes
CNN.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))

# "After ReLU comes a pooling step, in which the CNN downsamples the convolved feature
# (to save on processing time), reducing the number of dimensions of the feature map,
# while still preserving the most critical feature information." - class notes

# "size of the max-pooling filter is typically 2x2 pixels" - from class notes
# the class notes talk about using a stride of 2, so I will use that here as well:
CNN.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
CNN.add(Dropout(1 - .9))

CNN.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNN.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2) )
CNN.add(Dropout(1 - .9))

CNN.add(layers.Flatten())
CNN.add(layers.Dense(512, activation='relu'))
CNN.add(Dropout(1 - .9))
# "Typically, the final fully-connected layer contains a softmax activation function,
# which outputs a probability value from 0 to 1 for each of the classification labels
# the model is trying to predict" - class notes
CNN.add(layers.Dense(10, activation='softmax')) 
#CNN.summary()

### "Use the training set to train your neural networks. Evaluate their performance on the validation data set."

###"After trying several different architectures, choose the one that performs best on the validation set."

-- instructions from professor

In [105]:
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)  # default values: RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
CNN.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])

CNN.fit(x=training_set, y=train_labels, batch_size=32, epochs=5, verbose=2, validation_split=0.5, shuffle=False)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
 - 13s - loss: 1.6135 - acc: 0.4199 - val_loss: 1.3733 - val_acc: 0.5032
Epoch 2/5
 - 10s - loss: 1.2147 - acc: 0.5723 - val_loss: 1.2109 - val_acc: 0.5788
Epoch 3/5
 - 10s - loss: 1.0335 - acc: 0.6413 - val_loss: 1.1303 - val_acc: 0.6168
Epoch 4/5
 - 10s - loss: 0.9032 - acc: 0.6909 - val_loss: 1.1673 - val_acc: 0.6172
Epoch 5/5
 - 10s - loss: 0.7851 - acc: 0.7336 - val_loss: 1.0436 - val_acc: 0.6493


### "Train this final architecture on the data from the training set and validation set and evaluate its performance on the test set."

In [106]:
CNN.fit(x=training_set, y=train_labels, batch_size=32, epochs=5, verbose=2, shuffle=True)
score = CNN.evaluate(test_set, test_labels, batch_size=32)
print()
print(str(score[1] * 100) + "% accuracy")

Epoch 1/5
 - 15s - loss: 0.9025 - acc: 0.6932
Epoch 2/5
 - 15s - loss: 0.8138 - acc: 0.7237
Epoch 3/5
 - 15s - loss: 0.7522 - acc: 0.7475
Epoch 4/5
 - 15s - loss: 0.7039 - acc: 0.7665
Epoch 5/5
 - 15s - loss: 0.6802 - acc: 0.7769
10000/10000 [==============================] - 1s 129us/step

68.19% accuracy
